# 🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Author : Laurent*

*Date : 24/06/2025*

**Description :**

This Jupyter Notebook compares distances between waypoints and bus stops using different methods:
1. Pre-calculated distances from the C2C SQL dump
2. Distances calculated using routingpy with OpenRouteService API (by foot and by car)

It uses pandas/geopandas to parse waypoints from the C2C CSV export for Isère `Liste_iti_D4G_isere.csv` 
and extract bus stop locations from a PostgreSQL SQL C2C dump `dump-c2corg-202505050900.sql`.


In [ ]:
import binascii
import re
import struct
from time import sleep

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import Point

# Set display options
pd.set_option("display.max_columns", None)

## 1. Load and Parse Bus Stop Data from SQL Dump


In [ ]:
def parse_sql_dump_stopareas(sql_file):
    """
    Parse the SQL dump to extract bus stops (stopareas) with their coordinates.

    Args:
        sql_file: Path to the SQL dump file

    Returns:
        GeoDataFrame with bus stop information
    """
    # Pattern to match INSERT statements for stopareas
    insert_pattern = re.compile(r"INSERT INTO guidebook\.stopareas VALUES (.*?);", re.DOTALL)

    # List to store parsed stop areas
    stopareas = []

    with open(sql_file, "r", encoding="utf-8") as f:
        content = f.read()

    # Process each INSERT statement
    for match in insert_pattern.finditer(content):
        values_str = match.group(1)

        # Split values preserving quoted strings (handling escaped quotes)
        values = []
        current = ""
        in_quote = False
        escape_next = False

        for char in values_str:
            if escape_next:
                current += char
                escape_next = False
                continue

            if char == "\\":
                escape_next = True
                continue

            if char == "'":
                in_quote = not in_quote
                continue

            if char == "," and not in_quote:
                values.append(current.strip())
                current = ""
                continue

            current += char

        if current.strip():
            values.append(current.strip())

        # Ensure we have enough values (at least 6)
        if len(values) < 6:
            continue

        try:
            # Extract basic fields
            stoparea_id = int(values[0].strip("("))
            navitia_id = values[1].strip("'")
            stoparea_name = values[2].strip("'")
            line = values[3].strip("'")
            operator = values[4].strip("'")
            ewkb_hex = values[5].strip("')")

            # Skip if geometry is NULL
            if ewkb_hex.upper() == "NULL":
                continue

            # Convert hex to binary
            try:
                ewkb_bytes = binascii.unhexlify(ewkb_hex)
            except binascii.Error:
                continue

            # Check if we have enough data
            if len(ewkb_bytes) < 17:  # Minimum for point without SRID
                continue

            # Read byte order (1 = little, 0 = big)
            byte_order = ewkb_bytes[0]
            is_little_endian = byte_order == 1

            # Read geometry type (4 bytes)
            geom_type_bytes = ewkb_bytes[1:5]

            # Extract geometry type (first 4 bits) and flags
            if is_little_endian:
                geom_type = struct.unpack("<I", geom_type_bytes)[0]
            else:
                geom_type = struct.unpack(">I", geom_type_bytes)[0]

            # Check if this is a point (type = 1) - ignore flags
            if (geom_type & 0x07) != 1:  # Use bitmask to ignore SRID flag and others
                continue

            # Read SRID if present (bit 3 of geom_type is set)
            offset = 5
            srid = None
            if geom_type & 0x20000000:  # Check if SRID flag is set
                if len(ewkb_bytes) < 9:
                    continue
                if is_little_endian:
                    srid = struct.unpack("<I", ewkb_bytes[5:9])[0]
                else:
                    srid = struct.unpack(">I", ewkb_bytes[5:9])[0]
                offset = 9

            # Read coordinates
            if len(ewkb_bytes) < offset + 16:  # 8 bytes for x, 8 for y
                continue

            # Extract X and Y coordinates
            if is_little_endian:
                x = struct.unpack("<d", ewkb_bytes[offset : offset + 8])[0]
                y = struct.unpack("<d", ewkb_bytes[offset + 8 : offset + 16])[0]
            else:
                x = struct.unpack(">d", ewkb_bytes[offset : offset + 8])[0]
                y = struct.unpack(">d", ewkb_bytes[offset + 8 : offset + 16])[0]

            # Add to list
            stopareas.append(
                {
                    "stoparea_id": stoparea_id,
                    "navitia_id": navitia_id,
                    "stoparea_name": stoparea_name,
                    "line": line,
                    "operator": operator,
                    "srid": srid,
                    "geometry": Point(x, y),
                }
            )

        except Exception as e:
            # Skip problematic rows
            print(f"Error parsing row: {e}")
            continue

    # Create GeoDataFrame
    if stopareas:
        gdf = gpd.GeoDataFrame(stopareas, geometry="geometry", crs="EPSG:3857")
        return gdf
    else:
        return gpd.GeoDataFrame([])

def parse_sql_dump_waypoints_stopareas(sql_file):
    """
    Parse the SQL dump to extract waypoints_stopareas with their distances.

    Args:
        sql_file: Path to the SQL dump file

    Returns:
        DataFrame with waypoint-stoparea distances
    """
    # Pattern to match INSERT statements for waypoints_stopareas
    insert_pattern = re.compile(r"INSERT INTO guidebook\.waypoints_stopareas VALUES (.*?);", re.DOTALL)

    # List to store parsed waypoints_stopareas
    waypoints_stopareas = []

    with open(sql_file, "r", encoding="utf-8") as f:
        content = f.read()

    # Process each INSERT statement
    for match in insert_pattern.finditer(content):
        values_str = match.group(1)

        # Split values preserving quoted strings (handling escaped quotes)
        values = []
        current = ""
        in_quote = False
        escape_next = False

        for char in values_str:
            if escape_next:
                current += char
                escape_next = False
                continue

            if char == "\\":
                escape_next = True
                continue

            if char == "'":
                in_quote = not in_quote
                continue

            if char == "," and not in_quote:
                values.append(current.strip())
                current = ""
                continue

            current += char

        if current.strip():
            values.append(current.strip())

        # Ensure we have enough values (at least 4)
        if len(values) < 4:
            continue

        try:
            # Extract fields
            stoparea_id = int(values[0].strip("("))
            document_id = int(values[1])
            waypoint_id = int(values[2])
            distance = float(values[3].strip(")"))

            # Add to list
            waypoints_stopareas.append(
                {
                    "waypoint_id": waypoint_id,
                    "document_id": document_id,
                    "stoparea_id": stoparea_id,
                    "distance": distance,
                }
            )

        except Exception as e:
            # Skip problematic rows
            print(f"Error parsing waypoints_stopareas row: {e}")
            continue

    # Create DataFrame
    if waypoints_stopareas:
        df = pd.DataFrame(waypoints_stopareas)
        return df
    else:
        return pd.DataFrame([])

# Load bus stops and waypoints_stopareas from SQL dump
sql_file_path = "../data/C2C/dump-c2corg-202505050900.sql"

try:
    # Extract bus stops
    stops_gdf = parse_sql_dump_stopareas(sql_file_path)
    print(f"Loaded {len(stops_gdf)} bus stops from SQL dump")

    if not stops_gdf.empty:
        print("First 2 bus stops:")
        print(stops_gdf.head(2).T)

    # Extract waypoints_stopareas
    waypoints_stopareas_df = parse_sql_dump_waypoints_stopareas(sql_file_path)
    print(f"Loaded {len(waypoints_stopareas_df)} waypoint-stoparea distances from SQL dump")

    if not waypoints_stopareas_df.empty:
        print("First 2 waypoint-stoparea distances:")
        print(waypoints_stopareas_df.head(5).T)

except FileNotFoundError:
    print(f"Error: SQL file '{sql_file_path}' not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


## 2. Load and Parse Waypoint Data from CSV


In [ ]:
# Read CSV file
csv_path = "../data/C2C/Liste_iti_D4G_isere.csv"
df = pd.read_csv(csv_path, on_bad_lines="skip", low_memory=False)

# Display sample data
print("First 2 rows of original CSV:")
print(df.head(2).T)

# Simplified pattern that focuses only on [X, Y] coordinates
coord_pattern = re.compile(r"\s*([+-]?\d+[\.\d]*)\s*,\s*([+-]?\d+[\.\d]*)\s*")

# List to store parsed waypoints
waypoints = []

# Identify waypoint columns (columns 9 onward)
waypoint_cols = df.columns[9:]

# Parse waypoints
for idx, row in df.iterrows():
    itinerary_id = row["Id itinéraire"]

    for col in waypoint_cols:
        cell = str(row[col])
        if not cell or cell == "nan":
            continue

        # Extract waypoint ID from cell content instead of column name
        waypoint_id = None
        # Look for pattern like "(12345 - Title - [X, Y])"
        # Using search instead of match to find the pattern anywhere in the cell
        id_match = re.search(r"\(\s*(\d+)\s*-", cell)
        if id_match:
            try:
                waypoint_id = int(id_match.group(1))
            except (ValueError, IndexError):
                pass

        match = coord_pattern.search(cell)
        if match:
            try:
                x = float(match.group(1))
                y = float(match.group(2))

                waypoint_data = {
                    "itinerary_id": itinerary_id,
                    "geometry": Point(x, y)
                }

                if waypoint_id:
                    waypoint_data["waypoint_id"] = waypoint_id

                waypoints.append(waypoint_data)

            except (ValueError, TypeError):
                continue  # Skip invalid coordinates

# Create GeoDataFrame
if waypoints:
    wp_gdf = gpd.GeoDataFrame(pd.DataFrame(waypoints), geometry="geometry", crs="EPSG:3857")
    print(f"\n✅ Successfully parsed {len(wp_gdf)} waypoints")
else:
    print("🚨 No coordinates found. Check if any cell contains [X, Y] format.")


## 3. Match Waypoints with Pre-calculated Distances


In [ ]:
# If we have waypoint IDs in both datasets, we can match them
if 'waypoint_id' in wp_gdf.columns and not waypoints_stopareas_df.empty:
    # Merge waypoints with waypoints_stopareas
    waypoints_with_distances = pd.merge(
        wp_gdf,
        waypoints_stopareas_df,
        on='waypoint_id',
        how='inner'
    )

    # Merge with stops to get stop coordinates
    waypoints_stops_distances = pd.merge(
        waypoints_with_distances,
        stops_gdf[['stoparea_id', 'stoparea_name', 'geometry']],
        on='stoparea_id',
        how='inner',
        suffixes=('_waypoint', '_stoparea')
    )

    # Remove duplicates having the same itinerary_id, waypoint_id, and distance
    original_count = len(waypoints_stops_distances)
    waypoints_stops_distances = waypoints_stops_distances.drop_duplicates(
        subset=['stoparea_id', 'waypoint_id', 'distance']
    )
    removed_count = original_count - len(waypoints_stops_distances)

    print(f"Found {original_count} waypoint-stoparea pairs with pre-calculated distances")
    print(f"Removed {removed_count} duplicates with the same itinerary_id, waypoint_id, and distance")
    print(f"Remaining {len(waypoints_stops_distances)} unique waypoint-stoparea pairs")

    if not waypoints_stops_distances.empty:
        print("Sample of waypoint-stoparea pairs:")
        print(waypoints_stops_distances[['waypoint_id', 'stoparea_id', 'stoparea_name', 'distance']].head())
else:
    print("No waypoint IDs found in both datasets, can't match pre-calculated distances")

    # For demonstration, we'll select a few waypoints and stops to calculate distances
    # In a real scenario, you might want to use a different approach
    sample_waypoints = wp_gdf.head(5)
    sample_stops = stops_gdf.head(5)

    print(f"Selected {len(sample_waypoints)} sample waypoints and {len(sample_stops)} sample stops for distance calculation")

    # Create a DataFrame with all combinations of waypoints and stops
    waypoints_stops_pairs = []

    for wp_idx, wp_row in sample_waypoints.iterrows():
        for stop_idx, stop_row in sample_stops.iterrows():
            waypoints_stops_pairs.append({
                'waypoint_geometry': wp_row.geometry,
                'stoparea_id': stop_row.stoparea_id,
                'stoparea_name': stop_row.stoparea_name,
                'stoparea_geometry': stop_row.geometry
            })

    waypoints_stops_distances = pd.DataFrame(waypoints_stops_pairs)

    print(f"Created {len(waypoints_stops_distances)} waypoint-stoparea pairs for distance calculation")


## 4. Calculate Distances using routingpy with OpenRouteService API


In [ ]:
from routingpy import ORS
from pyproj import Transformer
import math

# Function to convert EPSG:3857 to EPSG:4326 (lat/lon)
def convert_to_latlon(point):
    """Convert a point from EPSG:3857 to EPSG:4326 (lat/lon)"""
    transformer = Transformer.from_crs('EPSG:3857', 'EPSG:4326', always_xy=True)
    lon, lat = transformer.transform(point.x, point.y)
    return lat, lon

# Function to calculate straight-line distance using Haversine formula
def calculate_haversine_distance(point1, point2):
    """
    Calculate the great-circle distance between two points on Earth using the Haversine formula.

    Args:
        point1: Tuple of (latitude, longitude) for the first point
        point2: Tuple of (latitude, longitude) for the second point

    Returns:
        Distance in kilometers
    """
    # Earth radius in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(point1[0])
    lon1 = math.radians(point1[1])
    lat2 = math.radians(point2[0])
    lon2 = math.radians(point2[1])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

# Set up OpenRouteService client
# You need to get an API key from https://openrouteservice.org/dev/#/signup
# Replace 'your_api_key' with your actual API key
ors_api_key = 'xxx'  # Replace with your actual API key
ors_client = ORS(api_key=ors_api_key)

# Function to calculate route distance between two points
def calculate_route_distance(start_point, end_point, profile='foot-walking'):
    """
    Calculate the route distance between two points using OpenRouteService.

    Args:
        start_point: Tuple of (latitude, longitude) for the start point
        end_point: Tuple of (latitude, longitude) for the end point
        profile: Routing profile ('foot-walking' or 'driving-car')

    Returns:
        Distance in kilometers
    """
    try:
        # Get route
        route = ors_client.directions(
            locations=[
                [start_point[1], start_point[0]],  # [lon, lat] for ORS
                [end_point[1], end_point[0]]
            ],
            profile=profile,
            format='geojson'
        )

        # Extract distance in kilometers
        distance_km = route.distance / 1000

        return distance_km

    except Exception as e:
        print(f"Error calculating route: {e}")
        return None

# Calculate distances for each waypoint-stoparea pair
# Note: This will make API calls, which might be rate-limited
# For demonstration, we'll calculate for a small sample
sample_size = min(100, len(waypoints_stops_distances))
# Sort by distance in descending order to get the pairs with the largest distances first
# if 'distance' in waypoints_stops_distances.columns:
#     waypoints_stops_distances = waypoints_stops_distances.sort_values(by='distance', ascending=False)
# sample_pairs = waypoints_stops_distances.head(sample_size)
sample_pairs = waypoints_stops_distances.sample(sample_size)

results = []

for idx, row in sample_pairs.iterrows():
    # Get waypoint and stoparea geometries
    if 'geometry_waypoint' in row:
        waypoint_geom = row.geometry_waypoint
        stoparea_geom = row.geometry_stoparea
    else:
        waypoint_geom = row.waypoint_geometry
        stoparea_geom = row.stoparea_geometry

    # Convert to lat/lon
    waypoint_latlon = convert_to_latlon(waypoint_geom)
    stoparea_latlon = convert_to_latlon(stoparea_geom)

    # Get pre-calculated distance if available
    precalculated_distance = row.get('distance', None)

    # Calculate straight-line distance using Haversine formula
    straight_line_distance = calculate_haversine_distance(waypoint_latlon, stoparea_latlon)

    # Calculate route distances
    # Note: use the API key
    sleep(4)
    foot_distance = calculate_route_distance(waypoint_latlon, stoparea_latlon, profile='foot-walking')
    car_distance = calculate_route_distance(waypoint_latlon, stoparea_latlon, profile='driving-car')

    # For demonstration, we'll use dummy values
    # foot_distance = precalculated_distance * 0.9 if precalculated_distance else None
    # car_distance = precalculated_distance * 1.1 if precalculated_distance else None

    results.append({
        'waypoint_latlon': waypoint_latlon,
        'itinerary_id': row.itinerary_id,
        'stoparea_id': row.stoparea_id,
        'stoparea_name': row.stoparea_name,
        'stoparea_latlon': stoparea_latlon,
        'precalculated_distance': precalculated_distance,
        'foot_distance': foot_distance,
        'car_distance': car_distance,
        'straight_line_distance': straight_line_distance
    })

# Create DataFrame with results
results_df = pd.DataFrame(results)

print("Distance calculation results:")
print(results_df[['stoparea_id', 'itinerary_id', 'precalculated_distance', 'straight_line_distance', 'foot_distance', 'car_distance']])


## 5. Visualize Distance Comparisons


In [ ]:
# Prepare data for visualization
if not results_df.empty:
    # Remove rows where any of the distance metrics is 0
    original_count = len(results_df)
    results_df = results_df[
        (results_df['foot_distance'] != 0) &
        (results_df['car_distance'] != 0) &
        (results_df['straight_line_distance'] != 0) &
        (results_df['precalculated_distance'] != 0)
    ]
    removed_count = original_count - len(results_df)
    print(f"Removed {removed_count} rows with zero distances")
    print(f"Remaining {len(results_df)} rows")

    # Melt the DataFrame to have distance type as a variable
    plot_data = pd.melt(
        results_df,
        id_vars=['stoparea_name'],
        value_vars=['precalculated_distance', 'foot_distance', 'car_distance', 'straight_line_distance'],
        var_name='distance_type',
        value_name='distance_km'
    )

    # Replace distance_type values with more readable labels
    plot_data['distance_type'] = plot_data['distance_type'].replace({
        'precalculated_distance': 'Pre-calculated',
        'foot_distance': 'Walking (ORS)',
        'car_distance': 'Driving (ORS)',
        'straight_line_distance': 'Straight-line (Haversine)'
    })

    # Create a grouped bar chart
    plt.figure(figsize=(12, 6))

    # Plot
    ax = plt.subplot(111)
    for i, distance_type in enumerate(['Pre-calculated', 'Walking (ORS)', 'Driving (ORS)', 'Straight-line (Haversine)']):
        data = plot_data[plot_data['distance_type'] == distance_type]
        x = np.arange(len(data))
        width = 0.2  # Reduced width to fit 4 bars
        ax.bar(x + i*width - 0.3, data['distance_km'], width, label=distance_type)

    # Add labels and legend
    ax.set_xlabel('Bus Stop')
    ax.set_ylabel('Distance (km)')
    ax.set_title('Distance Comparison: Pre-calculated vs. OpenRouteService')
    ax.set_xticks(np.arange(len(results_df)))
    ax.set_xticklabels(results_df['stoparea_name'], rotation=45, ha='right')
    ax.legend()

    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization")


## 6. Summary and Conclusions


In [ ]:
# Calculate statistics
if not results_df.empty:
    # Calculate differences between pre-calculated and calculated distances
    if 'precalculated_distance' in results_df.columns:
        #
        # # After calculating the mean values
        # straight_line_diff_pct_mean = results_df['straight_line_diff_pct'].mean()
        # foot_diff_pct_mean = results_df['foot_diff_pct'].mean()
        # car_diff_pct_mean = results_df['car_diff_pct'].mean()
        #
        # # Calculate absolute percentage differences to determine which is closest
        # abs_straight_line_diff = abs(straight_line_diff_pct_mean)
        # abs_foot_diff = abs(foot_diff_pct_mean)
        # abs_car_diff = abs(car_diff_pct_mean)
        #
        # # Determine which distance type is closest to precalculated value
        # closest_type = "car" if abs_car_diff < abs_foot_diff else "foot"
        # closest_diff = min(abs_car_diff, abs_foot_diff)
        #
        # # Conclusions
        # print("\nConclusions:")
        # print("1. The pre-calculated distances in the SQL dump are consistently larger than all other calculated distances (straight-line, walking, and driving).")
        # print(f"2. Straight-line distances show the largest difference, being on average {abs_straight_line_diff:.1f}% shorter than pre-calculated distances.")
        # print(f"3. Walking distances are on average {abs_foot_diff:.1f}% shorter than pre-calculated distances.")
        # print(f"4. Car distances are on average {abs_car_diff:.1f}% shorter than pre-calculated distances.")
        # print(f"5. {closest_type.capitalize()} distances are closest to the pre-calculated values, with only {closest_diff:.1f}% difference.")
        # print("6. The significant differences suggest that the pre-calculated distances may be using different routing algorithms, include additional factors, or potentially overestimate the actual distances.")
        #
        #



        #
        # # Differences with pre-calculated distance
        # results_df['foot_diff'] = results_df['precalculated_distance'] - results_df['foot_distance']
        # results_df['car_diff'] =  results_df['precalculated_distance'] - results_df['car_distance']
        # results_df['straight_line_diff'] = results_df['precalculated_distance'] - results_df['straight_line_distance']
        results_df['foot_diff'] = abs(results_df['foot_distance'] - results_df['precalculated_distance'])
        results_df['car_diff'] =  abs(results_df['car_distance'] - results_df['precalculated_distance'])
        results_df['straight_line_diff'] = abs(results_df['straight_line_distance'] - results_df['precalculated_distance'])
        #
        # # Percentage differences with pre-calculated distance
        # results_df['foot_diff_pct'] = (results_df['foot_diff'] / results_df['precalculated_distance']) * 100
        # results_df['car_diff_pct'] = (results_df['car_diff'] / results_df['precalculated_distance']) * 100
        # results_df['straight_line_diff_pct'] = (results_df['straight_line_diff'] / results_df['precalculated_distance']) * 100
        #
        # Summary statistics
        print("Summary Statistics:")
        print("\nAbsolute Differences from Pre-calculated Distance (km):")
        print(results_df[['straight_line_diff', 'foot_diff', 'car_diff']].describe())
        #
        # print("\nPercentage Differences from Pre-calculated Distance (%):")
        # print(results_df[['straight_line_diff_pct', 'foot_diff_pct', 'car_diff_pct']].describe())
        #
        # # Calculate ratio of precalc to route distances
        # results_df['precalc_to_foot_ratio'] = results_df['precalculated_distance'] / results_df['foot_distance']
        # results_df['precalc_to_car_ratio'] = results_df['precalculated_distance'] / results_df['car_distance']
        #
        # print("\nRatio of precalc to Route Distances:")
        # print(results_df[['precalc_to_foot_ratio', 'precalc_to_car_ratio']].describe())
        #
        # # Get mean values from the statistics
        # straight_line_diff_pct_mean = results_df['straight_line_diff_pct'].mean()
        # foot_diff_pct_mean = results_df['foot_diff_pct'].mean()
        # car_diff_pct_mean = results_df['car_diff_pct'].mean()
        #
        # # Conclusions
        # print("\nConclusions:")
        # print("1. The pre-calculated distances in the SQL dump are consistently larger than all other calculated distances (straight-line, walking, and driving).")
        # print(f"2. Straight-line distances show the largest difference, being on average {abs(straight_line_diff_pct_mean):.1f}% shorter than pre-calculated distances.")
        # print(f"3. Walking distances are on average {abs(foot_diff_pct_mean):.1f}% shorter than pre-calculated distances.")
        # print(f"4. Car distances are on average {abs(car_diff_pct_mean):.1f}% shorter than pre-calculated distances, making them closest to the pre-calculated values.")
        # print("5. The significant differences suggest that the pre-calculated distances may be using different routing algorithms, include additional factors, or potentially overestimate the actual distances.")
else:
    print("No data available for analysis")
